In [5]:
from keras import models, layers, optimizers, regularizers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

#import warnings
#warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Data Preprocessing

In [6]:
train = pd.read_csv('stockTrainSet.csv')
test = pd.read_csv('stockTestSet.csv')

## Set target data
Set target as 

1: if next day's close price is "higher" than today's close price

0: if next day's close price is "lower" than today's close price

In [7]:
def addTarget(df):
    target = np.zeros(len(df))
    for i in range(len(df)-1):
        cur = df.iloc[i, 2]
        nxt = df.iloc[i+1, 2]

        if nxt >= cur:
            target[i] = 1
        else:
            target[i] = 0

    df['target'] = target

In [8]:
addTarget(train)
addTarget(test)

train.describe()

,Open Price,Close Price,High Price,Low Price,Volume,target
count,2264.000000,2264.000000,2264.000000,2264.000000,2.264000e+03,2264.000000
mean,1656.142686,1656.767562,1664.427054,1647.425128,2.948755e+09,0.547261
std,485.292193,485.226120,484.808706,485.576116,1.351675e+09,0.497871
min,679.280000,676.530000,695.270000,666.790000,5.181584e+08,0.000000
25%,1238.602500,1239.337500,1246.695000,1227.587500,2.048980e+09,0.000000
50%,1649.135000,1650.405000,1656.145000,1639.600000,2.506637e+09,1.000000
75%,2070.300000,2071.190000,2079.507500,2058.757500,3.373334e+09,1.000000
max,2692.710000,2690.160000,2694.970000,2685.920000,9.120100e+09,1.000000


## Feature Selection
- Open Price
- Close Price	
- High Price	
- Low Price
- Volume


In [9]:
train_x = train.drop(columns=["Date", "target"])
train_y = train["target"]
#train_y = np.array(train_y, dtype=int)

test_x = test.drop(columns=["Date", "target"])
test_y = test["target"]
#test_y = np.array(test_y, dtype=int)

## Normalization
Since different features may use different scales, normalization can help learning algorithms be more effective

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_x)

train_x_std = scaler.transform(train_x)
test_x_std = scaler.transform(test_x)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys


# Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
_ = lr_model.fit(train_x_std, train_y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
lr_predict = lr_model.predict(test_x_std)
print(accuracy_score(test_y, lr_predict))

0.5238095238095238


## Improve my classifier <a class="tocSkip">

## SGD Classifier
When given the parameter loss="log", SGD Classifier is a
incrementally trained logistic regression.

In [13]:
from sklearn import linear_model

SGDlr_model = linear_model.SGDClassifier(loss='log',
                                         max_iter=1000,
                                         tol=1e-3)
_ = SGDlr_model.fit(train_x_std, train_y)

In [14]:
SGDlr_predict = SGDlr_model.predict(test_x_std)
print(accuracy_score(test_y, SGDlr_predict))

0.4801587301587302


# SVM(Support Vector Machine)

In [15]:
from sklearn.svm import SVC

svm_model = SVC()
_ = svm_model.fit(train_x_std, train_y)

The accuracy is very low also.

In [16]:
svm_predict = svm_model.predict(test_x_std)
print(accuracy_score(test_y, svm_predict))

0.5238095238095238


## Improve my classifier <a class="tocSkip">
## Change kernel to sigmoid
The result didn't change, it's still 0.5238

In [17]:
svm_model = SVC(kernel="sigmoid")
_ = svm_model.fit(train_x_std, train_y)

svm_predict = svm_model.predict(test_x_std)
print(accuracy_score(test_y, svm_predict))

0.5238095238095238


## Change kernel to poly
The result is still same

In [18]:
svm_model = SVC(kernel="poly")
_ = svm_model.fit(train_x_std, train_y)

svm_predict = svm_model.predict(test_x_std)
print(accuracy_score(test_y, svm_predict))

0.5238095238095238


## Change degree to 4
The result is still same

In [21]:
svm_model = SVC(kernel="poly", degree=4)
_ = svm_model.fit(train_x_std, train_y)

svm_predict = svm_model.predict(test_x_std)
print(accuracy_score(test_y, svm_predict))

0.5238095238095238


# Neural Network
Construct a neueal network without any hidden layer, and choose relu as activation function

In [30]:
model = models.Sequential()
model.add(layers.Dense(units=1,
                       input_dim=train_x.shape[1], 
                       activation='relu'))

In [31]:
model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=0.01))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 6         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [34]:
history = model.fit(train_x_std, 
                    train_y, 
                    epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
2264/2264 [==============================] - 0s 79us/step - loss: 3.3196
Epoch 2/10
2264/2264 [==============================] - 0s 22us/step - loss: 1.0112
Epoch 3/10
2264/2264 [==============================] - 0s 20us/step - loss: 0.7084
Epoch 4/10
2264/2264 [==============================] - 0s 20us/step - loss: 0.7025
Epoch 5/10
2264/2264 [==============================] - 0s 22us/step - loss: 0.6986
Epoch 6/10
2264/2264 [==============================] - 0s 21us/step - loss: 0.6959
Epoch 7/10
2264/2264 [==============================] - 0s 22us/step - loss: 0.6948
Epoch 8/10
2264/2264 [==============================] - 0s 24us/step - loss: 0.6928
Epoch 9/10
2264/2264 [==============================] - 0s 22us/step - loss: 0.6925
Epoch 10/10
2264/2264 [==============================] - 0s 22us/step - loss: 0.6918


In [37]:
train_acc = model.evaluate(train_x_std, train_y, batch_size=32)
test_acc = model.evaluate(test_x_std, test_y, batch_size=32)

252/252 [==============================] - 0s 15us/step


0.6914061588027873

In [29]:
nn_pred = model.predict_classes(test_x_std)
print(accuracy_score(test_y, nn_pred))

0.5238095238095238


## Improve my classifier <a class="tocSkip">
## Add two hidden layers

In [38]:
model = models.Sequential()
model.add(layers.Dense(units=10,
                       input_dim=train_x.shape[1], 
                       activation='relu'))

model.add(layers.Dense(units=10,
                       activation='relu'))

model.add(layers.Dense(units=1,
                       activation='sigmoid'))


model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=0.01))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________


In [39]:
nn_pred = model.predict_classes(test_x_std)
print(accuracy_score(test_y, nn_pred))

0.5238095238095238


# Another Data set

In [41]:
df = pd.read_csv('titanicTrainSet.csv')
df = df.drop(['PassengerId','Name', 'Ticket', 'Cabin', 'Embarked','Fare'], axis = 1)

age = df['Age']
avgage = age.sum(axis = 0, skipna = True) /age.count()
df['Age'].fillna(avgage, inplace = True)

sex_map = {'male' : 0, 'female' :1}
df['Sex'] = df['Sex'].map(sex_map)

In [42]:
train_x = df[:700].drop('Survived', 1)
train_y = df['Survived'][:700]

test_x = df[700:].drop('Survived', 1)
test_y = df['Survived'][700:]

### normalization

In [43]:
scaler = StandardScaler()
scaler.fit(train_x)

train_x_std = scaler.transform(train_x)
test_x_std = scaler.transform(test_x)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


## Logistic Regression

In [44]:
lr_model = LogisticRegression(class_weight='balanced', solver='liblinear')#have adjust parameter
lr_model.fit(train_x_std, train_y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [45]:
lr_predict = lr_model.predict(test_x_std)
print(accuracy_score(test_y, lr_predict))

0.8167539267015707


## SVM

In [138]:
svm_model = SVC()
svm_model.fit(train_x_std, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [178]:
svm_predict = svm_model.predict(test_x_std)
print(accuracy_score(test_y, svm_predict))

0.8534031413612565


## Neural Network

In [175]:
model = models.Sequential()
model.add(layers.Dense(units=10,
                       input_dim=train_x.shape[1], 
                       activation='relu'))

model.add(layers.Dense(units=10, 
                       activation='relu'))

model.add(layers.Dense(units=1, 
                       activation='sigmoid'))

In [176]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 10)                70        
_________________________________________________________________
dense_27 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_28 (Dense)             (None, 1)                 11        
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________


In [179]:
history = model.fit(train_x_std, 
                    train_y, 
                    epochs=10)

train_acc = model.evaluate(train_x_std, train_y)
test_acc = model.evaluate(test_x_std, test_y)

nn_pred = model.predict_classes(test_x_std)
print(accuracy_score(test_y, nn_pred))

Epoch 1/10
700/700 [==============================] - 0s 43us/step - loss: 0.4245
Epoch 2/10
700/700 [==============================] - 0s 44us/step - loss: 0.4238
Epoch 3/10
700/700 [==============================] - 0s 45us/step - loss: 0.4229
Epoch 4/10
700/700 [==============================] - 0s 41us/step - loss: 0.4222
Epoch 5/10
700/700 [==============================] - 0s 51us/step - loss: 0.4242
Epoch 6/10
700/700 [==============================] - 0s 47us/step - loss: 0.4226
Epoch 7/10
700/700 [==============================] - 0s 45us/step - loss: 0.4213
Epoch 8/10
700/700 [==============================] - 0s 44us/step - loss: 0.4200
Epoch 9/10
700/700 [==============================] - 0s 48us/step - loss: 0.4223
Epoch 10/10
191/191 [==============================] - 0s 31us/step
0.8586387434554974


# Discussion

## Which classifier reaches the highest classification accuracy in this dataset ?


### Why?

### Can this result remain if the dataset is different ?


## How did you improve your classifiers ?
